# Day 7: Custom Tools - Building Your Own AI Superpowers

## What You'll Learn Today

Welcome to Day 7! Today you'll learn how to create **custom tools** that extend what your agents can do!

**What's a Custom Tool?**
Think of tools as **abilities** you give to your AI agent:
- A **weather tool** lets the agent check the weather
- A **calculator tool** lets the agent do exact calculations
- A **database tool** lets the agent query data
- An **image generation tool** lets the agent create images

### Today's Learning Path:
1. **The @register_tool decorator** - Registering tools globally
2. **Parameter schemas** - Defining what inputs your tool accepts
3. **JSON Schema deep dive** - Types, nested objects, enums, arrays
4. **Real tool examples** - Weather API, calculator, database query
5. **json5 parsing** - Handling tool arguments
6. **Tool registry mechanism** - How Qwen-Agent manages tools
7. **Advanced patterns** - Stateful tools, async tools
8. **Tool testing** - Strategies for testing your tools

Let's build some powerful tools! 🛠️

---
## Part 1: Configure Our Environment

Same Fireworks API setup as previous days.

In [ ]:
# ================================================
# FIREWORKS API CONFIGURATION
# ================================================
import os
import json

# Set API credentials
os.environ['FIREWORKS_API_KEY'] = 'fw_3ZSpUnVR78vs38jJtyewjcWk'

# Standard configuration for Fireworks Qwen3-235B-A22B-Thinking
llm_cfg_fireworks = {
    'model': 'accounts/fireworks/models/qwen3-235b-a22b-thinking-2507',
    'model_server': 'https://api.fireworks.ai/inference/v1',
    'api_key': os.environ['FIREWORKS_API_KEY'],
    'generate_cfg': {
        'max_tokens': 32768,
        'temperature': 0.6,
    }
}

# Use this as default llm_cfg
llm_cfg = llm_cfg_fireworks

print('✅ Configured for Fireworks API')
print(f'   Model: Qwen3-235B-A22B-Thinking-2507')
print(f'   Max tokens: 32,768')

✅ Configured for Fireworks API
   Model: Qwen3-235B-A22B-Thinking-2507
   Max tokens: 32,768

---
## Part 2: Understanding Custom Tools

### What is a Tool?

A **tool** is a Python class that:
1. Inherits from `BaseTool`
2. Defines what it does (`description`)
3. Defines what parameters it needs (`parameters`)
4. Implements the actual logic (`call()` method)

### The BaseTool Structure

```python
class BaseTool:
    name: str                    # Tool identifier
    description: str             # What the tool does
    parameters: List[Dict]       # What inputs it needs
    
    def call(self, params: str, **kwargs) -> str:
        # Your implementation
        pass
```

**Think of it like this:**
- `name` = The tool's name in the toolbox
- `description` = Instructions on when to use it
- `parameters` = The dials and knobs on the tool
- `call()` = What happens when you use the tool

Let's create your first custom tool!

---
## Part 3: Your First Custom Tool - Simple Calculator

### Creating a Basic Tool

Let's create a calculator tool that can add two numbers.

In [ ]:
from qwen_agent.tools.base import BaseTool, register_tool
import json5

@register_tool('simple_calculator')
class SimpleCalculator(BaseTool):
    """
    A simple calculator that adds two numbers.
    
    The @register_tool decorator registers this tool globally
    so agents can use it by name: 'simple_calculator'
    """
    
    # What the tool does (shown to the LLM)
    description = 'A calculator that adds two numbers together'
    
    # What parameters it needs (JSON Schema format)
    parameters = [
        {
            'name': 'a',
            'type': 'number',
            'description': 'The first number',
            'required': True
        },
        {
            'name': 'b',
            'type': 'number',
            'description': 'The second number',
            'required': True
        }
    ]
    
    def call(self, params: str, **kwargs) -> str:
        """
        The actual implementation.
        
        Args:
            params: JSON string with parameters (e.g., '{"a": 5, "b": 3}')
            **kwargs: Additional context (messages, etc.)
            
        Returns:
            Result as a string (LLMs work with text!)
        """
        # Parse the JSON parameters using json5 (more forgiving than json)
        args = json5.loads(params)
        
        # Extract the parameters
        a = args['a']
        b = args['b']
        
        # Do the calculation
        result = a + b
        
        # Return as string (important!)
        return f"The sum of {a} and {b} is {result}"

print("✅ SimpleCalculator tool registered!")
print(f"   Tool name: simple_calculator")
print(f"   Description: {SimpleCalculator.description}")
print(f"   Parameters: {len(SimpleCalculator.parameters)} params")

✅ SimpleCalculator tool registered!
   Tool name: simple_calculator
   Description: A calculator that adds two numbers together
   Parameters: 2 params

### Testing the Tool Directly

Before using it with an agent, let's test it directly:

In [ ]:
# Create an instance of the tool
calc = SimpleCalculator()

# Test it with different inputs
test_cases = [
    '{"a": 10, "b": 5}',
    '{"a": 100, "b": 200}',
    '{"a": -5, "b": 12}',
]

print("Testing SimpleCalculator:\n")
for test in test_cases:
    result = calc.call(test)
    print(f"Input: {test}")
    print(f"Output: {result}\n")

Testing SimpleCalculator:

Input: {"a": 10, "b": 5}
Output: The sum of 10 and 5 is 15

Input: {"a": 100, "b": 200}
Output: The sum of 100 and 200 is 300

Input: {"a": -5, "b": 12}
Output: The sum of -5 and 12 is 7

### Using the Tool with an Agent

Now let's give this tool to an agent:

In [ ]:
from qwen_agent.agents import Assistant

# Create an agent with our custom tool
calculator_agent = Assistant(
    llm=llm_cfg,
    name='Calculator Agent',
    function_list=['simple_calculator'],  # Use the registered name
    system_message='You are a helpful calculator. Use the simple_calculator tool to add numbers.'
)

# Test it
messages = [{'role': 'user', 'content': 'What is 42 + 58?'}]

print("User: What is 42 + 58?\n")
for response in calculator_agent.run(messages):
    if response:
        last = response[-1]
        if last.get('function_call'):
            print(f"[Calling tool: {last['function_call']['name']}]")
            print(f"[Arguments: {last['function_call']['arguments']}]\n")
        elif last.get('role') == 'assistant' and last.get('content'):
            print(f"Assistant: {last['content']}")

ValidationError: 1 validation error for FunctionCall
arguments
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type


---
## Part 4: The @register_tool Decorator

### How @register_tool Works

The `@register_tool` decorator does several things:

1. **Registers the tool globally** - Makes it available by name
2. **Sets the tool name** - The string you pass becomes `tool.name`
3. **Adds it to the registry** - So agents can find it

### Syntax:

```python
@register_tool('my_tool_name')  # The name agents will use
class MyTool(BaseTool):
    # Your tool implementation
    pass
```

### Without @register_tool

You can also use tools without registering them (pass the class directly):

In [ ]:
# Define a tool WITHOUT @register_tool
class MultiplyTool(BaseTool):
    name = 'multiply'  # Must set name manually!
    description = 'Multiplies two numbers'
    parameters = [
        {'name': 'x', 'type': 'number', 'description': 'First number', 'required': True},
        {'name': 'y', 'type': 'number', 'description': 'Second number', 'required': True}
    ]
    
    def call(self, params: str, **kwargs) -> str:
        args = json5.loads(params)
        result = args['x'] * args['y']
        return f"{args['x']} × {args['y']} = {result}"

# Use it by passing the class directly
agent_with_multiply = Assistant(
    llm=llm_cfg,
    function_list=[MultiplyTool()],  # Pass an instance
)

print("✅ Created agent with MultiplyTool (not registered globally)")

✅ Created agent with MultiplyTool (not registered globally)

### When to Use Each Approach

| Approach | When to Use |
|----------|-------------|
| **@register_tool** | Tool will be reused across many agents |
| **Direct class** | Tool is specific to one agent |
| **Direct instance** | Tool needs initialization parameters |

---
## Part 5: Parameter Schemas - JSON Schema Deep Dive

### Understanding JSON Schema

The `parameters` list defines what inputs your tool accepts. It uses JSON Schema format.

### Basic Types

JSON Schema supports these types:

```python
# String
{'name': 'message', 'type': 'string', 'description': 'A text message'}

# Number (int or float)
{'name': 'age', 'type': 'number', 'description': 'Age in years'}

# Integer (whole numbers only)
{'name': 'count', 'type': 'integer', 'description': 'Item count'}

# Boolean
{'name': 'is_active', 'type': 'boolean', 'description': 'Active status'}

# Array
{'name': 'items', 'type': 'array', 'description': 'List of items'}

# Object (nested structure)
{'name': 'config', 'type': 'object', 'description': 'Configuration object'}
```

Let's see examples of each!

### Example 1: String Parameters with Enums

In [ ]:
@register_tool('weather_api')
class WeatherAPI(BaseTool):
    """
    A mock weather API tool demonstrating string parameters and enums.
    """
    description = 'Get weather information for a city'
    
    parameters = [
        {
            'name': 'city',
            'type': 'string',
            'description': 'The city name (e.g., "San Francisco")',
            'required': True
        },
        {
            'name': 'units',
            'type': 'string',
            'description': 'Temperature units',
            'enum': ['celsius', 'fahrenheit'],  # Only these values allowed!
            'required': False
        }
    ]
    
    def call(self, params: str, **kwargs) -> str:
        args = json5.loads(params)
        city = args['city']
        units = args.get('units', 'celsius')  # Default to celsius
        
        # Mock weather data
        temp = 22 if units == 'celsius' else 72
        symbol = '°C' if units == 'celsius' else '°F'
        
        return json.dumps({
            'city': city,
            'temperature': temp,
            'units': symbol,
            'condition': 'Sunny'
        }, ensure_ascii=False)

# Test it
weather = WeatherAPI()
print("Testing WeatherAPI:\n")
print(weather.call('{"city": "Tokyo", "units": "celsius"}'))
print(weather.call('{"city": "New York", "units": "fahrenheit"}'))

Testing WeatherAPI:

{"city": "Tokyo", "temperature": 22, "units": "°C", "condition": "Sunny"}
{"city": "New York", "temperature": 72, "units": "°F", "condition": "Sunny"}

### Example 2: Array Parameters

In [ ]:
@register_tool('batch_calculator')
class BatchCalculator(BaseTool):
    """
    A calculator that works on arrays of numbers.
    """
    description = 'Calculate statistics for a list of numbers'
    
    parameters = [
        {
            'name': 'numbers',
            'type': 'array',
            'description': 'Array of numbers to analyze',
            'items': {'type': 'number'},  # Each item must be a number
            'required': True
        },
        {
            'name': 'operation',
            'type': 'string',
            'description': 'Operation to perform',
            'enum': ['sum', 'average', 'min', 'max'],
            'required': True
        }
    ]
    
    def call(self, params: str, **kwargs) -> str:
        args = json5.loads(params)
        numbers = args['numbers']
        operation = args['operation']
        
        if operation == 'sum':
            result = sum(numbers)
        elif operation == 'average':
            result = sum(numbers) / len(numbers)
        elif operation == 'min':
            result = min(numbers)
        elif operation == 'max':
            result = max(numbers)
        
        return f"The {operation} of {numbers} is {result}"

# Test it
batch_calc = BatchCalculator()
print("Testing BatchCalculator:\n")
print(batch_calc.call('{"numbers": [1, 2, 3, 4, 5], "operation": "sum"}'))
print(batch_calc.call('{"numbers": [10, 20, 30], "operation": "average"}'))

Testing BatchCalculator:

The sum of [1, 2, 3, 4, 5] is 15
The average of [10, 20, 30] is 20.0

### Example 3: Nested Objects

In [ ]:
@register_tool('database_query')
class DatabaseQuery(BaseTool):
    """
    A mock database query tool with nested parameters.
    """
    description = 'Query a database with filters'
    
    parameters = [
        {
            'name': 'table',
            'type': 'string',
            'description': 'Table name to query',
            'required': True
        },
        {
            'name': 'filters',
            'type': 'object',
            'description': 'Filter conditions',
            'properties': {
                'age': {
                    'type': 'integer',
                    'description': 'Filter by age'
                },
                'city': {
                    'type': 'string',
                    'description': 'Filter by city'
                },
                'active': {
                    'type': 'boolean',
                    'description': 'Filter by active status'
                }
            },
            'required': False
        },
        {
            'name': 'limit',
            'type': 'integer',
            'description': 'Maximum number of results',
            'required': False
        }
    ]
    
    def call(self, params: str, **kwargs) -> str:
        args = json5.loads(params)
        table = args['table']
        filters = args.get('filters', {})
        limit = args.get('limit', 10)
        
        # Mock query result
        query = f"SELECT * FROM {table}"
        if filters:
            conditions = [f"{k}={v}" for k, v in filters.items()]
            query += " WHERE " + " AND ".join(conditions)
        query += f" LIMIT {limit}"
        
        return json.dumps({
            'query': query,
            'result_count': 5,
            'message': 'Query executed successfully'
        }, ensure_ascii=False)

# Test it
db = DatabaseQuery()
print("Testing DatabaseQuery:\n")
result1 = db.call('{"table": "users", "filters": {"age": 25, "city": "Tokyo"}, "limit": 5}')
print(json.dumps(json.loads(result1), indent=2))

Testing DatabaseQuery:

{
  "query": "SELECT * FROM users WHERE age=25 AND city=Tokyo LIMIT 5",
  "result_count": 5,
  "message": "Query executed successfully"
}

---
## Part 6: Real-World Tool Example - Image Generation

### Building the Image Generation Tool from Official Examples

Let's recreate the image generation tool from `assistant_add_custom_tool.py`:

In [ ]:
import urllib.parse

@register_tool('my_image_gen')
class MyImageGen(BaseTool):
    """
    AI painting (image generation) service.
    Uses the Pollinations.ai free API.
    """
    description = 'AI painting (image generation) service, input text description, and return the image URL drawn based on text information.'
    
    parameters = [
        {
            'name': 'prompt',
            'type': 'string',
            'description': 'Detailed description of the desired image content, in English',
            'required': True,
        }
    ]

    def call(self, params: str, **kwargs) -> str:
        # Parse parameters
        prompt = json5.loads(params)['prompt']
        
        # URL-encode the prompt
        prompt = urllib.parse.quote(prompt)
        
        # Return the image URL
        return json.dumps(
            {'image_url': f'https://image.pollinations.ai/prompt/{prompt}'},
            ensure_ascii=False,
        )

# Test it directly
image_gen = MyImageGen()
result = image_gen.call('{"prompt": "a beautiful sunset over mountains"}')
print("Testing MyImageGen:\n")
print(result)

# Parse and display the URL
url_data = json.loads(result)
print(f"\nGenerated image URL: {url_data['image_url']}")

Testing MyImageGen:

{"image_url": "https://image.pollinations.ai/prompt/a%20beautiful%20sunset%20over%20mountains"}

Generated image URL: https://image.pollinations.ai/prompt/a%20beautiful%20sunset%20over%20mountains

### Using Image Generation with an Agent

In [ ]:
# Create an AI painting agent
painter_agent = Assistant(
    llm=llm_cfg,
    name='AI Painter',
    description='AI painting service',
    system_message='You are an AI artist. When users request images, use my_image_gen to create them. Describe what you created.',
    function_list=['my_image_gen'],
)

# Test it
messages = [{'role': 'user', 'content': 'Draw a picture of a cat sitting on a laptop'}]

print("User: Draw a picture of a cat sitting on a laptop\n")
for response in painter_agent.run(messages):
    if response:
        last = response[-1]
        if last.get('function_call'):
            print(f"[Calling: {last['function_call']['name']}]")
            print(f"[Prompt: {json5.loads(last['function_call']['arguments'])['prompt']}]\n")
        elif last.get('role') == 'assistant' and last.get('content'):
            print(f"AI Painter: {last['content']}")

ValidationError: 1 validation error for FunctionCall
arguments
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type


---
## Part 7: Understanding json5 Parsing

### Why json5 Instead of json?

**json5** is more forgiving than standard JSON:

```python
# Standard JSON (strict)
import json
json.loads('{"key": "value"}')  # OK
json.loads("{key: 'value'}")     # ERROR! Single quotes not allowed
json.loads('{key: "value"}')    # ERROR! Unquoted keys not allowed

# JSON5 (forgiving)
import json5
json5.loads('{"key": "value"}')  # OK
json5.loads("{key: 'value'}")     # OK! Single quotes allowed
json5.loads('{key: "value"}')    # OK! Unquoted keys allowed
json5.loads('{key: "value",}')  # OK! Trailing commas allowed
```

### LLMs Sometimes Generate Imperfect JSON

When LLMs generate function arguments, they might:
- Use single quotes instead of double quotes
- Leave keys unquoted
- Add trailing commas
- Add comments

**json5 handles all of these gracefully!**

In [ ]:
import json
import json5

# Examples that would fail with json but work with json5
tricky_inputs = [
    "{key: 'value'}",              # Single quotes + unquoted key
    '{"a": 1, "b": 2,}',           # Trailing comma
    '{number: 42}',                # Unquoted key
    """{/* comment */ x: 10}""",  # With comment
]

print("Testing json5 vs json:\n")
for i, test_input in enumerate(tricky_inputs, 1):
    print(f"Test {i}: {test_input}")
    
    # Try with standard json
    try:
        result = json.loads(test_input)
        print(f"  json: ✅ {result}")
    except Exception as e:
        print(f"  json: ❌ {type(e).__name__}")
    
    # Try with json5
    try:
        result = json5.loads(test_input)
        print(f"  json5: ✅ {result}")
    except Exception as e:
        print(f"  json5: ❌ {type(e).__name__}")
    
    print()

Testing json5 vs json:

Test 1: {key: 'value'}
  json: ❌ JSONDecodeError
  json5: ✅ {'key': 'value'}

Test 2: {"a": 1, "b": 2,}
  json: ❌ JSONDecodeError
  json5: ✅ {'a': 1, 'b': 2}

Test 3: {number: 42}
  json: ❌ JSONDecodeError
  json5: ✅ {'number': 42}

Test 4: {/* comment */ x: 10}
  json: ❌ JSONDecodeError
  json5: ✅ {'x': 10}

### Best Practice: Always Use json5 in Tool Implementation

```python
def call(self, params: str, **kwargs) -> str:
    # ✅ Good: Use json5
    args = json5.loads(params)
    
    # ❌ Bad: Use json (might fail with LLM-generated args)
    # args = json.loads(params)
```

---
## Part 8: Tool Registry Mechanism

### How Qwen-Agent Manages Tools

When you use `@register_tool`, it adds your tool to a **global registry**.

### The Registry Process:

```
@register_tool('my_tool')
     ↓
Tool is added to TOOL_REGISTRY
     ↓
Agent asks for 'my_tool'
     ↓
Registry returns the tool class
     ↓
Agent instantiates and uses it
```

### Viewing Registered Tools

In [ ]:
from qwen_agent.tools import TOOL_REGISTRY

print("All registered tools:\n")
for tool_name in sorted(TOOL_REGISTRY.keys()):
    tool_class = TOOL_REGISTRY[tool_name]
    description = getattr(tool_class, 'description', 'No description')
    print(f"  • {tool_name}")
    print(f"    {description[:80]}..." if len(description) > 80 else f"    {description}")
    print()

All registered tools:

  • amap_weather
    获取对应城市的天气数据

  • batch_calculator
    Calculate statistics for a list of numbers

  • code_interpreter
    Python code sandbox, which can be used to execute Python code.

  • database_query
    Query a database with filters

  • doc_parser
    对一个文件进行内容提取和分块、返回分块后的文件内容

  • extract_doc_vocabulary
    提取文档的词表。

  • front_page_search
    从给定文档中检索和问题相关的部分

  • hybrid_search
    从给定文档中检索和问题相关的部分

  • image_gen
    An image generation service that takes text descriptions as input and returns a ...

  • image_search
    Image search engine, input the image and search for similar images with image in...

  • image_zoom_in_tool
    Zoom in on a specific region of an image by cropping it based on a bounding box ...

  • keyword_search
    从给定文档中检索和问题相关的部分

  • my_image_gen
    AI painting (image generation) service, input text description, and return the i...

  • retrieval
    从给定文件列表中检索出和问题相关的内容，支持文件类型包括：pdf / docx / pptx / txt / html / csv / tsv / 

### Three Ways to Provide Tools to Agents

```python
# Method 1: By name (must be registered)
agent = Assistant(
    function_list=['my_tool', 'code_interpreter']
)

# Method 2: By class (doesn't need registration)
agent = Assistant(
    function_list=[MyToolClass, AnotherToolClass]
)

# Method 3: By instance (for tools with config)
agent = Assistant(
    function_list=[MyToolClass(config='value')]
)

# Method 4: Mix and match!
agent = Assistant(
    function_list=[
        'code_interpreter',      # By name
        MyToolClass,             # By class
        ConfiguredTool(x=10),    # By instance
    ]
)
```

---
## Part 9: Advanced Patterns - Stateful Tools

### What is a Stateful Tool?

A **stateful tool** remembers information between calls.

Example use cases:
- A counter that increments
- A shopping cart that accumulates items
- A session manager that tracks state

In [ ]:
@register_tool('counter')
class Counter(BaseTool):
    """
    A stateful counter tool that remembers its count.
    """
    description = 'A counter that can increment, decrement, or reset'
    
    parameters = [
        {
            'name': 'action',
            'type': 'string',
            'description': 'Action to perform',
            'enum': ['increment', 'decrement', 'reset', 'get'],
            'required': True
        }
    ]
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.count = 0  # State!
    
    def call(self, params: str, **kwargs) -> str:
        args = json5.loads(params)
        action = args['action']
        
        if action == 'increment':
            self.count += 1
            return f"Incremented. Count is now {self.count}"
        elif action == 'decrement':
            self.count -= 1
            return f"Decremented. Count is now {self.count}"
        elif action == 'reset':
            self.count = 0
            return "Counter reset to 0"
        elif action == 'get':
            return f"Current count: {self.count}"

# Test the stateful tool
counter = Counter()
print("Testing stateful Counter:\n")
print(counter.call('{"action": "get"}'))
print(counter.call('{"action": "increment"}'))
print(counter.call('{"action": "increment"}'))
print(counter.call('{"action": "increment"}'))
print(counter.call('{"action": "get"}'))
print(counter.call('{"action": "reset"}'))
print(counter.call('{"action": "get"}'))

Testing stateful Counter:

Current count: 0
Incremented. Count is now 1
Incremented. Count is now 2
Incremented. Count is now 3
Current count: 3
Counter reset to 0
Current count: 0

---
## Part 10: Tool Testing Strategies

### Strategy 1: Unit Testing Tools Directly

Test tools without using an agent:

In [ ]:
def test_tool_directly():
    """
    Test a tool by calling it directly with various inputs.
    """
    tool = SimpleCalculator()
    
    # Test case 1: Normal addition
    result = tool.call('{"a": 10, "b": 5}')
    assert "15" in result, f"Expected 15 in result, got {result}"
    print("✅ Test 1 passed: Normal addition")
    
    # Test case 2: Negative numbers
    result = tool.call('{"a": -5, "b": 10}')
    assert "5" in result, f"Expected 5 in result, got {result}"
    print("✅ Test 2 passed: Negative numbers")
    
    # Test case 3: Floats
    result = tool.call('{"a": 1.5, "b": 2.5}')
    assert "4" in result or "4.0" in result, f"Expected 4 in result, got {result}"
    print("✅ Test 3 passed: Float numbers")
    
    print("\n✅ All tests passed!")

test_tool_directly()

✅ Test 1 passed: Normal addition
✅ Test 2 passed: Negative numbers
✅ Test 3 passed: Float numbers

✅ All tests passed!

### Strategy 2: Testing with Mock Agents

Test how your tool integrates with agents:

In [ ]:
def test_tool_with_agent():
    """
    Test a tool integrated with an agent.
    """
    agent = Assistant(
        llm=llm_cfg,
        function_list=['simple_calculator'],
        system_message='Use the calculator for all math.'
    )
    
    messages = [{'role': 'user', 'content': 'What is 25 plus 17?'}]
    
    # Run the agent
    final_response = None
    for response in agent.run(messages):
        final_response = response
    
    # Check that tool was called
    tool_was_called = any(
        msg.get('function_call', {}).get('name') == 'simple_calculator'
        for msg in final_response
    )
    
    assert tool_was_called, "Tool was not called by agent"
    print("✅ Agent successfully called the tool")
    
    # Check final answer contains expected result
    final_content = final_response[-1].get('content', '')
    assert '42' in final_content, f"Expected 42 in answer, got: {final_content}"
    print("✅ Agent provided correct answer")

test_tool_with_agent()

ValidationError: 1 validation error for FunctionCall
arguments
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type


---
## Part 11: Practice Exercises

Now it's your turn! Try these exercises.

### Exercise 1: Create a Temperature Converter Tool

Create a tool that converts temperatures between Celsius, Fahrenheit, and Kelvin.

Requirements:
- Parameters: `value` (number), `from_unit` (enum), `to_unit` (enum)
- Units: 'celsius', 'fahrenheit', 'kelvin'
- Return the converted temperature

In [ ]:
# TODO: Implement TemperatureConverter
# Hint: Use enums for from_unit and to_unit
# Hint: Formulas:
#   C to F: (C * 9/5) + 32
#   F to C: (F - 32) * 5/9
#   C to K: C + 273.15
#   K to C: K - 273.15

# @register_tool('temperature_converter')
# class TemperatureConverter(BaseTool):
#     ...

### Exercise 2: Create a String Manipulator Tool

Create a tool that performs various string operations.

Requirements:
- Parameters: `text` (string), `operation` (enum)
- Operations: 'uppercase', 'lowercase', 'reverse', 'length', 'word_count'
- Return the result as a string

In [ ]:
# TODO: Implement StringManipulator
# @register_tool('string_manipulator')
# class StringManipulator(BaseTool):
#     ...

---
## Summary: What You Learned Today

### Core Concepts

✅ **Custom tools** - Extend agents with new capabilities

✅ **BaseTool structure** - name, description, parameters, call()

✅ **@register_tool** - Register tools globally by name

✅ **JSON Schema** - Define parameters with types, enums, arrays, objects

✅ **json5 parsing** - Handle LLM-generated JSON gracefully

✅ **Tool registry** - How Qwen-Agent manages tools

✅ **Stateful tools** - Tools that remember state between calls

✅ **Testing strategies** - Direct testing, integration testing, error handling

### Key Takeaways

1. **Tools are Python classes** - Inherit from BaseTool
2. **Three ways to provide tools** - By name, class, or instance
3. **JSON Schema is powerful** - Use enums, arrays, nested objects
4. **Always use json5** - More forgiving than json
5. **Test tools directly first** - Don't wait for agent integration
6. **Stateful tools are powerful** - But manage state carefully
7. **Document your tools well** - Good descriptions help LLMs use them correctly

### What's Next?

**Tomorrow (Day 8)**: We'll dive deep into the **Assistant Agent**!

You'll learn:
- Complete Assistant initialization
- File handling and automatic RAG
- System message engineering
- function_list variations
- Real-world assistant patterns

---

**Congratulations! 🎉 You can now create custom tools that give your agents superpowers!**